In [9]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


data_dir = Path("data/features/only features")
files = list(data_dir.glob("*.csv"))
stock_data = {f.stem.replace("_features", ""): pd.read_csv(f) for f in files}



def make_direction_labels(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["Date"] = pd.to_datetime(df["Date"], errors = "coerce")

    for d in [5,10,20]:
        ma_col = f"MA{d}"
        ma_next = df[ma_col].shift(-1)
        dir_col = f"MA{d}_dir"
        df[dir_col] = np.where(ma_next > df[ma_col], 1 , -1)

    df = df.iloc[:-1].reset_index(drop = True)
    return df

labeled_data = {ticker: make_direction_labels(df) for ticker, df in stock_data.items()}

In [2]:
# Now we need to split the data into training and testing sets
# But since we have 3 different targets, we need to split the data for each target
# And since this is the time series data, we need to split the data by the index, not the random split

targets_dir = ["MA5_dir", "MA10_dir", "MA20_dir"]

split_data = {}

for ticker, df in labeled_data.items():

    df = df.copy()
    df = df.sort_values("Date")


    exclude_cols = ["Date"] + targets_dir

    split_idx = int(len(df) * 0.8)

    X_all = df.drop(columns=exclude_cols, errors="ignore")  
    X_train_raw = X_all.iloc[:split_idx].copy()
    X_test_raw  = X_all.iloc[split_idx:].copy()

    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train_raw), 
                           columns=X_train_raw.columns, index=X_train_raw.index)
    X_test  = pd.DataFrame(scaler.transform(X_test_raw), 
                           columns=X_test_raw.columns, index=X_test_raw.index)

    split_data[ticker] = {"X_train": X_train, "X_test": X_test}
    for t in targets_dir:
        y_all = df[t]
        y_train = y_all.iloc[:split_idx]
        y_test  = y_all.iloc[split_idx:]
        split_data[ticker][t] = {"y_train": y_train, "y_test": y_test}

In [3]:
# Running the Decision Tree Regression
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

results= []
models = {}

for ticker, parts in split_data.items():
    X_train = parts["X_train"]
    X_test  = parts["X_test"]

    models[ticker] = {}

    for target in targets_dir:
        y_train = parts[target]["y_train"]
        y_test  = parts[target]["y_test"]

        reg = DecisionTreeRegressor(
            random_state=42,
            max_depth=None,
            min_samples_leaf=5,
            criterion="squared_error" 
        )

        reg.fit(X_train, y_train)

        y_pred_cont = reg.predict(X_test)
        y_pred = np.where(y_pred_cont > 0, 1, -1)  

        mae  = mean_absolute_error(y_test, y_pred)
        mse  = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2   = r2_score(y_test, y_pred)


        results.append({
            "Ticker": ticker,
            "Target": target.replace("_dir",""),  
            "Model": "DecisionTreeClassifier",
            "MAE": mae,                    
            "MSE": mse,
            "RMSE": rmse,
            "R2": r2
        })

        models[ticker][target] = reg

res_df = pd.DataFrame(results).sort_values(["Ticker", "Target"])

res_df["Target"] = pd.Categorical(res_df["Target"], categories=["MA5","MA10","MA20"], ordered=True)
res_df = res_df.sort_values(["Ticker","Target"])

table = (
    res_df
    .set_index(["Ticker","Target","Model"])
    [["MAE","MSE","RMSE","R2"]]
    .sort_index()
)

print(table.to_string())

                                           MAE       MSE      RMSE        R2
Ticker Target Model                                                         
AAPL   MA5    DecisionTreeClassifier  0.535211  1.070423  1.034612 -0.079470
       MA10   DecisionTreeClassifier  0.366197  0.732394  0.855800  0.258188
       MA20   DecisionTreeClassifier  0.295775  0.591549  0.769122  0.387931
ADBE   MA5    DecisionTreeClassifier  0.443662  0.887324  0.941979  0.112280
       MA10   DecisionTreeClassifier  0.345070  0.690141  0.830747  0.298589
       MA20   DecisionTreeClassifier  0.232394  0.464789  0.681754  0.499359
AMD    MA5    DecisionTreeClassifier  0.725352  1.450704  1.204452 -0.453299
       MA10   DecisionTreeClassifier  0.387324  0.774648  0.880141  0.224390
       MA20   DecisionTreeClassifier  0.140845  0.281690  0.530745  0.716906
CRM    MA5    DecisionTreeClassifier  0.436620  0.873239  0.934473  0.126067
       MA10   DecisionTreeClassifier  0.338028  0.676056  0.822226  0.320574

In [ ]:
# Running the SVM
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


svm_results = []
svm_models = {}

for ticker, parts in split_data.items():
    X_train = parts["X_train"]
    X_test  = parts["X_test"]

    svm_models[ticker] = {}

    for target in targets_dir:  # ["MA5_dir","MA10_dir","MA20_dir"]
        y_train = parts[target]["y_train"]
        y_test  = parts[target]["y_test"]

        svm = SVR(kernel="linear", C=1.0, epsilon=0.1)
        svm.fit(X_train, y_train)

        y_pred_cont = svm.predict(X_test)
        y_pred = np.where(y_pred_cont > 0, 1, -1)

        mae  = mean_absolute_error(y_test, y_pred)
        mse  = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2   = r2_score(y_test, y_pred)

        svm_results.append({
            "Ticker": ticker,
            "Target": target.replace("_dir",""), 
            "Model": "SVR",
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R2": r2
        })

        svm_models[ticker][target] = svm

svm_res_df = pd.DataFrame(svm_results)
svm_res_df["Target"] = pd.Categorical(svm_res_df["Target"], categories=["MA5","MA10","MA20"], ordered=True)
svm_table = (
    svm_res_df
    .set_index(["Ticker","Target","Model"])
    [["MAE","MSE","RMSE","R2"]]
    .sort_index()
)
print(svm_table.to_string())

                          MAE       MSE      RMSE        R2
Ticker Target Model                                        
AAPL   MA5    SVR    0.345070  0.690141  0.830747  0.304026
       MA10   SVR    0.281690  0.563380  0.750587  0.429375
       MA20   SVR    0.211268  0.422535  0.650027  0.562808
ADBE   MA5    SVR    0.288732  0.577465  0.759911  0.422277
       MA10   SVR    0.232394  0.464789  0.681754  0.527621
       MA20   SVR    0.330986  0.661972  0.813617  0.286966
AMD    MA5    SVR    0.394366  0.788732  0.888106  0.209857
       MA10   SVR    0.267606  0.535211  0.731581  0.464124
       MA20   SVR    0.176056  0.352113  0.593391  0.646132
CRM    MA5    SVR    0.316901  0.633803  0.796117  0.365694
       MA10   SVR    0.218310  0.436620  0.660772  0.561204
       MA20   SVR    0.302817  0.605634  0.778225  0.394336
MSFT   MA5    SVR    0.415493  0.830986  0.911584  0.144578
       MA10   SVR    0.253521  0.507042  0.712069  0.486438
       MA20   SVR    0.211268  0.422535 

In [ ]:
# Runnding Bagging 

from sklearn.ensemble import BaggingRegressor

bag_results = []
bag_models = {}

for ticker, parts in split_data.items():
    X_train = parts["X_train"]
    X_test  = parts["X_test"]

    bag_models[ticker] = {}

    for target in targets_dir:  # ["MA5_dir","MA10_dir","MA20_dir"]
        y_train = parts[target]["y_train"]
        y_test  = parts[target]["y_test"]

        base = DecisionTreeRegressor(random_state=42, min_samples_leaf=5)
        model = BaggingRegressor(
            estimator=base,        # scikit-learn >= 1.2
            n_estimators=300,
            bootstrap=True,
            n_jobs=-1,
            random_state=42
        )
        model.fit(X_train, y_train)

        y_pred_cont = model.predict(X_test)
        y_pred = np.where(y_pred_cont > 0, 1, -1)

        mae  = mean_absolute_error(y_test, y_pred)
        mse  = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2   = r2_score(y_test, y_pred)

        bag_results.append({
            "Ticker": ticker,
            "Target": target.replace("_dir",""),  # "MA5","MA10","MA20"
            "Model": "BaggingRegressor",
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R2": r2
        })

        bag_models[ticker][target] = model

bag_res_df = pd.DataFrame(bag_results)
bag_res_df["Target"] = pd.Categorical(bag_res_df["Target"], categories=["MA5","MA10","MA20"], ordered=True)
bag_table = (
    bag_res_df
    .set_index(["Ticker","Target","Model"])
    [["MAE","MSE","RMSE","R2"]]
    .sort_index()
)
print(bag_table.to_string())

                                     MAE       MSE      RMSE        R2
Ticker Target Model                                                   
AAPL   MA5    BaggingRegressor  0.359155  0.718310  0.847532  0.275619
       MA10   BaggingRegressor  0.274648  0.549296  0.741145  0.443641
       MA20   BaggingRegressor  0.211268  0.422535  0.650027  0.562808
ADBE   MA5    BaggingRegressor  0.302817  0.605634  0.778225  0.394096
       MA10   BaggingRegressor  0.232394  0.464789  0.681754  0.527621
       MA20   BaggingRegressor  0.161972  0.323944  0.569160  0.651068
AMD    MA5    BaggingRegressor  0.323944  0.647887  0.804914  0.350954
       MA10   BaggingRegressor  0.253521  0.507042  0.712069  0.492328
       MA20   BaggingRegressor  0.140845  0.281690  0.530745  0.716906
CRM    MA5    BaggingRegressor  0.359155  0.718310  0.847532  0.281120
       MA10   BaggingRegressor  0.204225  0.408451  0.639101  0.589514
       MA20   BaggingRegressor  0.197183  0.394366  0.627986  0.605614
MSFT  

In [ ]:
# Random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

rf_results = []
rf_models = {}

for ticker, parts in split_data.items():
    X_train = parts["X_train"]
    X_test  = parts["X_test"]

    rf_models[ticker] = {}

    for target in targets_dir:  # ["MA5_dir","MA10_dir","MA20_dir"]
        y_train = parts[target]["y_train"]
        y_test  = parts[target]["y_test"]

        model = RandomForestRegressor(
            n_estimators=300,
            min_samples_leaf=3,
            random_state=42,
            n_jobs=-1,
            bootstrap=True
        )
        model.fit(X_train, y_train)

        y_pred_cont = model.predict(X_test)
        y_pred = np.where(y_pred_cont > 0, 1, -1)

        mae  = mean_absolute_error(y_test, y_pred)
        mse  = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2   = r2_score(y_test, y_pred)

        rf_results.append({
            "Ticker": ticker,
            "Target": target.replace("_dir",""),  # "MA5","MA10","MA20"
            "Model": "RandomForestRegressor",
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R2": r2
        })

        rf_models[ticker][target] = model

rf_res_df = pd.DataFrame(rf_results)
rf_res_df["Target"] = pd.Categorical(rf_res_df["Target"], categories=["MA5","MA10","MA20"], ordered=True)
rf_table = (
    rf_res_df
    .set_index(["Ticker","Target","Model"])
    [["MAE","MSE","RMSE","R2"]]
    .sort_index()
)
print(rf_table.to_string())

                                          MAE       MSE      RMSE        R2
Ticker Target Model                                                        
AAPL   MA5    RandomForestRegressor  0.359155  0.718310  0.847532  0.275619
       MA10   RandomForestRegressor  0.281690  0.563380  0.750587  0.429375
       MA20   RandomForestRegressor  0.211268  0.422535  0.650027  0.562808
ADBE   MA5    RandomForestRegressor  0.338028  0.676056  0.822226  0.323642
       MA10   RandomForestRegressor  0.218310  0.436620  0.660772  0.556250
       MA20   RandomForestRegressor  0.183099  0.366197  0.605142  0.605556
AMD    MA5    RandomForestRegressor  0.323944  0.647887  0.804914  0.350954
       MA10   RandomForestRegressor  0.274648  0.549296  0.741145  0.450022
       MA20   RandomForestRegressor  0.147887  0.295775  0.543852  0.702751
CRM    MA5    RandomForestRegressor  0.366197  0.732394  0.855800  0.267024
       MA10   RandomForestRegressor  0.204225  0.408451  0.639101  0.589514
       MA20 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

ada_results = []
ada_models = {}

for ticker, parts in split_data.items():
    X_train = parts["X_train"]
    X_test  = parts["X_test"]

    ada_models[ticker] = {}

    for target in targets_dir:  # ["MA5_dir","MA10_dir","MA20_dir"]
        y_train = parts[target]["y_train"]
        y_test  = parts[target]["y_test"]

        weak = DecisionTreeRegressor(max_depth=3, random_state=42)

        # sklearn 버전 호환 (estimator vs base_estimator)
        try:
            model = AdaBoostRegressor(
                estimator=weak,           # scikit-learn >= 1.2
                n_estimators=300,
                learning_rate=0.1,
                random_state=42
            )
        except TypeError:
            model = AdaBoostRegressor(
                base_estimator=weak,      # scikit-learn < 1.2
                n_estimators=300,
                learning_rate=0.1,
                random_state=42
            )

        model.fit(X_train, y_train)

        y_pred_cont = model.predict(X_test)
        y_pred = np.where(y_pred_cont > 0, 1, -1)

        mae  = mean_absolute_error(y_test, y_pred)
        mse  = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2   = r2_score(y_test, y_pred)

        ada_results.append({
            "Ticker": ticker,
            "Target": target.replace("_dir",""),  # "MA5","MA10","MA20"
            "Model": "AdaBoostRegressor(Depth3Tree)",
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R2": r2
        })

        ada_models[ticker][target] = model

ada_res_df = pd.DataFrame(ada_results)
ada_res_df["Target"] = pd.Categorical(ada_res_df["Target"], categories=["MA5","MA10","MA20"], ordered=True)
ada_table = (
    ada_res_df
    .set_index(["Ticker","Target","Model"])
    [["MAE","MSE","RMSE","R2"]]
    .sort_index()
)
print(ada_table.to_string())


                                                  MAE       MSE      RMSE        R2
Ticker Target Model                                                                
AAPL   MA5    AdaBoostRegressor(Depth3Tree)  0.316901  0.633803  0.796117  0.360840
       MA10   AdaBoostRegressor(Depth3Tree)  0.295775  0.591549  0.769122  0.400844
       MA20   AdaBoostRegressor(Depth3Tree)  0.211268  0.422535  0.650027  0.562808
ADBE   MA5    AdaBoostRegressor(Depth3Tree)  0.288732  0.577465  0.759911  0.422277
       MA10   AdaBoostRegressor(Depth3Tree)  0.253521  0.507042  0.712069  0.484677
       MA20   AdaBoostRegressor(Depth3Tree)  0.253521  0.507042  0.712069  0.453846
AMD    MA5    AdaBoostRegressor(Depth3Tree)  0.309859  0.619718  0.787222  0.379173
       MA10   AdaBoostRegressor(Depth3Tree)  0.260563  0.521127  0.721891  0.478226
       MA20   AdaBoostRegressor(Depth3Tree)  0.119718  0.239437  0.489323  0.759370
CRM    MA5    AdaBoostRegressor(Depth3Tree)  0.309859  0.619718  0.787222  0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool


cat_results = []
cat_models = {}

for ticker, parts in split_data.items():
    X_train = parts["X_train"]
    X_test  = parts["X_test"]

    cat_models[ticker] = {}

    for target in targets_dir:  # ["MA5_dir","MA10_dir","MA20_dir"]
        y_train = parts[target]["y_train"]
        y_test  = parts[target]["y_test"]

        train_pool = Pool(X_train, y_train)
        test_pool  = Pool(X_test,  y_test)

        model = CatBoostRegressor(
            iterations=300,
            depth=6,
            learning_rate=0.1,
            loss_function="RMSE",
            random_seed=42,
            verbose=False,
            allow_writing_files=False,
            thread_count=-1
        )

        model.fit(train_pool)

        y_pred_cont = model.predict(test_pool)
        y_pred = np.where(y_pred_cont > 0, 1, -1)

        mae  = mean_absolute_error(y_test, y_pred)
        mse  = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2   = r2_score(y_test, y_pred)

        cat_results.append({
            "Ticker": ticker,
            "Target": target.replace("_dir",""),  # "MA5","MA10","MA20"
            "Model": "CatBoostRegressor",
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R2": r2
        })

        cat_models[ticker][target] = model

cat_res_df = pd.DataFrame(cat_results)
cat_res_df["Target"] = pd.Categorical(cat_res_df["Target"], categories=["MA5","MA10","MA20"], ordered=True)
cat_table = (
    cat_res_df
    .set_index(["Ticker","Target","Model"])
    [["MAE","MSE","RMSE","R2"]]
    .sort_index()
)
print(cat_table.to_string())

                                      MAE       MSE      RMSE        R2
Ticker Target Model                                                    
AAPL   MA5    CatBoostRegressor  0.352113  0.704225  0.839181  0.289822
       MA10   CatBoostRegressor  0.267606  0.535211  0.731581  0.457906
       MA20   CatBoostRegressor  0.260563  0.521127  0.721891  0.460796
ADBE   MA5    CatBoostRegressor  0.323944  0.647887  0.804914  0.351823
       MA10   CatBoostRegressor  0.260563  0.521127  0.721891  0.470363
       MA20   CatBoostRegressor  0.197183  0.394366  0.627986  0.575214
AMD    MA5    CatBoostRegressor  0.338028  0.676056  0.822226  0.322734
       MA10   CatBoostRegressor  0.288732  0.577465  0.759911  0.421818
       MA20   CatBoostRegressor  0.133803  0.267606  0.517306  0.731061
CRM    MA5    CatBoostRegressor  0.401408  0.802817  0.896001  0.196546
       MA10   CatBoostRegressor  0.211268  0.422535  0.650027  0.575359
       MA20   CatBoostRegressor  0.211268  0.422535  0.650027  0